# Train a deep learning model
In this notebook you will train a deep learning model to classify the descriptions of car components as compliant or non-compliant. 

Each document in the supplied training data set is a short text description of the component as documented by an authorized technician. 
The contents include:
- Manufacture year of the component (e.g. 1985, 2010)
- Condition of the component (poor, fair, good, new)
- Materials used in the component (plastic, carbon fiber, steel, iron)

The compliance regulations dictate:
*Any component manufactured before 1995 or in fair or poor condition or made with plastic or iron is out of compliance.*

For example:
* Manufactured in 1985 made of steel in fair condition -> **Non-compliant**
* Good condition carbon fiber component manufactured in 2010 -> **Compliant**
* Steel component manufactured in 1995 in fair condition -> **Non-Compliant**

The labels present in this data are 0 for compliant, 1 for non-compliant.

The challenge with classifying text data is that deep learning models only undertand vectors (e.g., arrays of numbers) and not text. To encode the car component descriptions as vectors, we use an algorithm from Stanford called [GloVe (Global Vectors for Word Representation)](https://nlp.stanford.edu/projects/glove/). GloVe provides us pre-trained vectors that we can use to convert a string of text into a vector.

### Setup
To begin, you will need to provide the following information about your Azure Subscription.

**If you are using your own Azure subscription, please provide names for subscription_id, resource_group, workspace_name and workspace_region to use.** Note that the workspace needs to be of type [Machine Learning Workspace](https://docs.microsoft.com/en-us/azure/machine-learning/service/setup-create-workspace).

**If an environment is provided to you be sure to replace XXXXX in the values below with your unique identifier.**

In the following cell, be sure to set the values for `subscription_id`, `resource_group`, `workspace_name` and `workspace_region` as directed by the comments (*these values can be acquired from the Azure Portal*).

To get these values, do the following:
1. Navigate to the Azure Portal and login with the credentials provided.
2. From the left hand menu, under Favorites, select `Resource Groups`.
3. In the list, select the resource group with the name similar to `XXXXX`.
4. From the Overview tab, capture the desired values.

Execute the following cell by selecting the `>|Run` button in the command bar above.

In [2]:
#Provide the Subscription ID of your existing Azure subscription
subscription_id = "6ee947fa-0d77-4915-bf68-4a83a8bec2a4" # <- needs to be the subscription with the Quick-Starts resource group

#Provide values for the existing Resource Group 
resource_group = "Quick-Starts-MLOps123" # <- replace XXXXX with your unique identifier

#Provide the Workspace Name and Azure Region of the Azure Machine Learning Workspace
workspace_name = "quick-starts-ws-mlops123" # <- replace XXXXX with your unique identifier
workspace_region = "northeurope" # <- region of your Quick-Starts resource group

In [3]:
experiment_name = 'deep-learning'
project_folder = './dl'
deployment_folder = './deploy'
datasets_folder = './datasets'

# this is the URL to the CSV file containing the car component descriptions
cardata_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
            'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/'
            'quickstarts/connected-car-data/connected-car_components.csv')

cardata_ds_name = 'connected_car_components'
cardata_ds_description = 'Connected car components data'

# this is the name of the AML Compute cluster
cluster_name = "gpucluster"

embedding_dim = 100                                        
training_samples = 90000                                 
validation_samples = 5000    
max_words = 10000

# Create the Azure Machine Learning resources

The Azure Machine Learning SDK provides a comprehensive set of a capabilities that you can use directly within a notebook including:
- Creating a **Workspace** that acts as the root object to organize all artifacts and resources used by Azure Machine Learning.
- Creating **Experiments** in your Workspace that capture versions of the trained model along with any desired model performance telemetry. Each time you train a model and evaluate its results, you can capture that run (model and telemetry) within an Experiment.
- Creating **Compute** resources that can be used to scale out model training, so that while your notebook may be running in a lightweight container in Azure Notebooks, your model training can actually occur on a powerful cluster that can provide large amounts of memory, CPU or GPU. 
- Using **Automated Machine Learning (AutoML)** to automatically train multiple versions of a model using a mix of different ways to prepare the data and different algorithms and hyperparameters (algorithm settings) in search of the model that performs best according to a performance metric that you specify.
- Packaging a Docker **Image** that contains everything your trained model needs for scoring (prediction) in order to run as a web service.
- Deploying your Image to either Azure Kubernetes or Azure Container Instances, effectively hosting the **Web Service**.

In Azure Notebooks, all of the libraries needed for Azure Machine Learning are pre-installed. To use them, you just need to import them. Run the following cell to do so:

In [4]:
import logging
import os
import json

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.model import Model
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.dnn import TensorFlow

from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [5]:
azureml.core.__version__

'1.0.72'

## Create and connect to an Azure Machine Learning Workspace
Run the following cell to create a new Azure Machine Learning **Workspace** and save the configuration to disk (next to the Jupyter notebook). 

**Important Note**: You will be prompted to login in the text that is output below the cell. Be sure to navigate to the URL displayed and enter the code that is provided. Once you have entered the code, return to this notebook and wait for the output to read `Workspace configuration succeeded`.

In [6]:
# By using the exist_ok param, if the worskpace already exists you get a reference to the existing workspace
# allowing you to re-run this cell multiple times as desired (which is fairly common in notebooks).
ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

ws.write_config()
print('Workspace configuration succeeded')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FKLZYLYSZ to authenticate.


WARNING - Failed to authenticate '{'additional_properties': {}, 'id': '/tenants/0f277086-d4e0-4971-bc1a-bbc5df0eb246', 'tenant_id': '0f277086-d4e0-4971-bc1a-bbc5df0eb246'}' due to error 'Get Token request returned http error: 400 and server response: {"error":"interaction_required","error_description":"AADSTS50076: Due to a configuration change made by your administrator, or because you moved to a new location, you must use multi-factor authentication to access '797f4846-ba00-4fd7-ba43-dac1f8f63013'.\r\nTrace ID: 7d4eb172-08b8-40ec-a603-f85b4b7e4e00\r\nCorrelation ID: 3d614027-f5ea-47aa-80a1-5335eb9611a4\r\nTimestamp: 2020-02-22 08:29:12Z","error_codes":[50076],"timestamp":"2020-02-22 08:29:12Z","trace_id":"7d4eb172-08b8-40ec-a603-f85b4b7e4e00","correlation_id":"3d614027-f5ea-47aa-80a1-5335eb9611a4","error_uri":"https://login.microsoftonline.com/error?code=50076","suberror":"basic_action"}'
WARNING - Failed to authenticate '{'additional_properties': {}, 'id': '/tenants/12437861-f55d-4e

Interactive authentication successfully completed.
Deploying StorageAccount with name quickstastorage957e4064b.
Deploying KeyVault with name quickstakeyvaulte73398e5.
Deployed KeyVault with name quickstakeyvaulte73398e5. Took 22.26 seconds.
Deploying AppInsights with name quickstainsightsd56b13cd.
Deployed AppInsights with name quickstainsightsd56b13cd. Took 28.83 seconds.
Deployed StorageAccount with name quickstastorage957e4064b. Took 56.92 seconds.
Deploying Workspace with name quick-starts-ws-mlops123.
Deployed Workspace with name quick-starts-ws-mlops123. Took 54.48 seconds.
Workspace configuration succeeded


### Create AML Compute Cluster
Now you are ready to create the GPU compute cluster. Run the following cell to create a new compute cluster (or retrieve the existing cluster if it already exists). The code below will create a *GPU based* cluster where each node in the cluster is of the size `Standard_NC12`, and the cluster is restricted to use 1 node. This will take couple of minutes to create.

In [7]:
### Create AML CPU based Compute Cluster

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC12',
                                                           min_nodes=1, max_nodes=1)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current AmlCompute. 
print(compute_target.status.serialize())

Creating a new compute target...
Creating
Succeeded..................
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 1, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2020-02-22T08:40:36.095000+00:00', 'errors': None, 'creationTime': '2020-02-22T08:40:35.184435+00:00', 'modifiedTime': '2020-02-22T08:41:21.487802+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': ''}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC12'}


### Create the Keras Estimator

In [8]:
keras_est = TensorFlow(source_directory=project_folder,
                       compute_target=compute_target,
                       framework_version='2.0',
                       entry_script='train.py',
                       conda_packages=['pandas'],
                       pip_packages=['azureml-dataprep[pandas]', 'keras==2.3.1'], # just add keras through pip
                       use_gpu=True)

## Remotely train a deep learning model using the Azure ML Compute
In the following cells, you will *not* train the model against the data you just downloaded using the resources provided by Azure Notebooks. Instead, you will deploy an Azure ML Compute cluster that will download the data and use a trainings script to train the model. In other words, all of the training will be performed remotely with respect to this notebook. 


In [9]:
# create project folder
if not os.path.exists(project_folder):
    os.makedirs(project_folder)

### Create the training script

In [10]:
%%writefile $project_folder/train.py

import os
import numpy as np
import pandas as pd

import keras
from keras import models 
from keras import layers
from keras import optimizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

from azureml.core import Run
from azureml.core.dataset import Dataset

datasets_folder = './datasets'

# this is the URL to the CSV file containing the GloVe vectors
glove_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
             'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/'
             'quickstarts/connected-car-data/glove.6B.100d.txt')

glove_ds_name = 'glove_6B_100d'
glove_ds_description ='GloVe embeddings 6B 100d'

# this is the URL to the CSV file containing the care component descriptions
cardata_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
            'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/'
            'quickstarts/connected-car-data/connected-car_components.csv')

cardata_ds_name = 'connected_car_components'
cardata_ds_description = 'Connected car components data'

embedding_dim = 100                                        
training_samples = 90000                                 
validation_samples = 5000    
max_words = 10000

run = Run.get_context()
ws = run.experiment.workspace

print("Downloading GloVe embeddings...")

try:
    glove_ds = Dataset.get_by_name(workspace=ws, name=glove_ds_name)
    print('GloVe embeddings dataset already registered.')
except:
    print('Registering GloVe embeddings dataset...')
    glove_ds = Dataset.File.from_files(glove_url)
    glove_ds.register(workspace=ws, name=glove_ds_name, description=glove_ds_description)
    print('GloVe embeddings dataset successfully registered.')
    
file_paths = glove_ds.download(target_path=datasets_folder, overwrite=True)
glove_file_path = file_paths[0]
print("Download complete.")


# Load the car components labeled data
print("Loading car components data...")

try:
    cardata_ds = Dataset.get_by_name(workspace=ws, name=cardata_ds_name)
    print('Connected car components dataset already registered.')
except:
    print('Registering connected car components dataset...')
    cardata_ds = Dataset.Tabular.from_delimited_files(path=cardata_url)
    cardata_ds.register(workspace=ws, name=cardata_ds_name, description=cardata_ds_description)
    print('Connected car components dataset successfully registered.')

car_components_df = cardata_ds.to_pandas_dataframe()
components = car_components_df["text"].tolist()
labels = car_components_df["label"].tolist()

print("Loading car components data completed.")


# use the Tokenizer from Keras to "learn" a vocabulary from the entire car components text
print("Tokenizing data...")    

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(components)
sequences = tokenizer.texts_to_sequences(components)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=embedding_dim)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])  
np.random.seed(100)
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]

x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

x_test = data[training_samples + validation_samples:]
y_test = labels[training_samples + validation_samples:]
print("Tokenizing data complete.")

# apply the vectors provided by GloVe to create a word embedding matrix
print("Applying GloVe vectors...")

embeddings_index = {}
f = open(glove_file_path)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector    
print("Applying GloVe vectors compelted.")

# use Keras to define the structure of the deep neural network   
print("Creating model structure...")

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=embedding_dim))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# fix the weights for the first layer to those provided by the embedding matrix
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
print("Creating model structure completed.")

opt = optimizers.RMSprop(lr=0.1)

print("Training model...")
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=3, 
                    batch_size=32,
                    validation_data=(x_val, y_val))
print("Training model completed.")

print("Saving model files...")
# create a ./outputs/model folder in the compute target
# files saved in the "./outputs" folder are automatically uploaded into run history
os.makedirs('./outputs/model', exist_ok=True)
# save model
model.save('./outputs/model/model.h5')
print("model saved in ./outputs/model folder")
print("Saving model files completed.")

Writing ./dl/train.py


## Submit the training run

The code pattern to submit a training run to Azure Machine Learning compute targets is always:

- Create an experiment to run.
- Submit the experiment.
- Wait for the run to complete.

### Create the experiment

In [11]:
experiment = Experiment(ws, experiment_name)

### Submit the experiment

In [12]:
run = experiment.submit(keras_est)

Wait for the run to complete by executing the following cell. Note that this process will perform the following:
- Build and deploy the container to Azure Machine Learning compute (~8 minutes)
- Execute the training script (~2 minutes)

If you change only the training script and re-submit, it will run faster the second time because the necessary container is already prepared so the time requried is just that for executing the training script.

*Run the cell below and wait till the **Run Status** is **Completed** before proceeding ahead*

In [14]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Download the model files from the run

In the training script, the Keras model is saved into two files, model.json and model.h5, in the outputs/models folder on the GPU cluster AmlCompute node. Azure ML automatically uploaded anything written in the ./outputs folder into run history file store. Subsequently, we can use the run object to download the model files. They are under the the outputs/model folder in the run history file store, and are downloaded into a local folder named model.

In [15]:
for f in run.get_file_names():
    print(f)

azureml-logs/20_image_build_log.txt
azureml-logs/55_azureml-execution-tvmps_3de1cc5e97e63143fd55036e1864e4122a4e82c21b700df4ae4bc7e16ee73a9e_d.txt
azureml-logs/65_job_prep-tvmps_3de1cc5e97e63143fd55036e1864e4122a4e82c21b700df4ae4bc7e16ee73a9e_d.txt
azureml-logs/70_driver_log.txt
azureml-logs/75_job_post-tvmps_3de1cc5e97e63143fd55036e1864e4122a4e82c21b700df4ae4bc7e16ee73a9e_d.txt
azureml-logs/process_info.json
azureml-logs/process_status.json
logs/azureml/128_azureml.log
logs/azureml/job_prep_azureml.log
logs/azureml/job_release_azureml.log
outputs/model/model.h5


In [16]:
# create a model folder in the current directory
os.makedirs('./model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)
        print('Download completed.')

Download completed.


## Restore the model from model.h5 file

In [17]:
model = load_model('./model/model.h5')
print("Model loaded from disk.")
print(model.summary())

WARNING - From /home/nbuser/anaconda3_501/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:182: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model loaded from disk.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                640064    
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 1,642,177
Trainable params: 642,177
Non-trainable params: 1,000,000
_________________________________________________________________
None


## Evaluate the model on test data

You can also evaluate how accurately the model performs against data it has not seen. Run the following cell to load the test data that was not used in either training or evaluating the model. 

In [18]:
# Load the car components labeled data
print("Loading car components data...")

cardata_ds = Dataset.get_by_name(workspace=ws, name=cardata_ds_name)
car_components_df = cardata_ds.to_pandas_dataframe()
components = car_components_df["text"].tolist()
labels = car_components_df["label"].tolist()

print("Loading car components data completed.")   

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(components)
sequences = tokenizer.texts_to_sequences(components)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=embedding_dim)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])      
np.random.seed(100)
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_test = data[training_samples + validation_samples:]
y_test = labels[training_samples + validation_samples:]

Loading car components data...
Loading car components data completed.
Found 65 unique tokens.
Shape of data tensor: (100000, 100)
Shape of label tensor: (100000,)


Run the following cell to see the accuracy on the test set (it is the second number in the array displayed, on a scale from 0 to 1).

In [19]:
print('Model evaluation will print the following metrics: ', model.metrics_names)
evaluation_metrics = model.evaluate(x_test, y_test)
print(evaluation_metrics)

Model evaluation will print the following metrics:  ['loss', 'acc']
5000/5000 [==============================] - 1s 264us/step
[0.7086361988067627, 0.49239999055862427]


Log the evaluation metrics to the to the experiment **Run**

In [20]:
run.log(model.metrics_names[0], evaluation_metrics[0], 'Model test data loss')
run.log(model.metrics_names[1], evaluation_metrics[1], 'Model test data accuracy')

**Save the run information to json file**

In [21]:
run_info = {}
run_info["id"] = run.id

print("Saving run_info.json...")
os.makedirs('./outputs', exist_ok=True)
run_info_filepath = os.path.join('./outputs', 'run_info.json')
with open(run_info_filepath, "w") as f:
    json.dump(run_info, f)

Saving run_info.json...
